In [60]:
'''
Title: Classifying Congressional Tweets Using Various Machine Learning Algorithms

Description:

I am trying to read tweets written by Congresspeople (Senate and House) and use machine learning to 
predict what party they're from. The tweets are collected using Twitter's API and the python library
Tweepy. I'm converting the text of the words to a bag of words, filtering out punctuation and stemming 
as well. I'm leaving in hashtags and @s. 

In terms of specific algorithms, I expect to use multiple different classifiers. I also will use a neural net,
though I'm not sure the architecture I'll aim for. I still need to do more research on what's commonly
used for NLP. It will likely incorporate existing sentiment analysis models to improve accuracy. 
In addition to that, I'll try a number of different classifiers, using grid search to optimize them, 
and test them with both test set accuracy and k-fold cross validation.

I plan to present the best models based on multiple metrics, including accuracy, time to learn, and time to
classify. I'll include the optimized parameters as well, and the optimal preparatory pipelines.

I've done most of the data preprocessing and collected a decent-sized dataset. I've run a logistic
regression classifier with default hyperparameters and a tfidf vectorizer. Stemming and filtering functions
are completed. You can see the accuracy of this basic model below; with 75% accuracy on the test set,
I'd say that's a good proof of concept.

I still need to complete my other models, and I also would like to get a larger dataset. I need to see how to
get more tweets per person using tweepy. I'll also look into other tools for preprocessing. And I'll
look into other dictionaries that could be helpful.

'''

"\nTitle: Classifying Congressional Tweets Using Various Machine Learning Algorithms\n\nDescription:\n\nI am trying to read tweets written by Congresspeople (Senate and House) and use machine learning to \npredict what party they're from. The tweets are collected using Twitter's API and the python library\nTweepy. I'm converting the text of the words to a bag of words, filtering out punctuation and stemming \nas well. I'm leaving in hashtags and @s. \n\nIn terms of specific algorithms, I expect to use multiple different classifiers. I also will use a neural net,\nthough I'm not sure the architecture I'll aim for. I still need to do more research on what's commonly\nused for NLP. It will likely incorporate existing sentiment analysis models to improve accuracy. \nIn addition to that, I'll try a number of different classifiers, using grid search to optimize them, \nand test them with both test set accuracy and k-fold cross validation.\n\nI plan to present the best models based on multipl

In [61]:
#test some stuff

import tweepy

auth = tweepy.AppAuthHandler("Y49bfRjEyvHDa0vUBRFC4oQJy", "CCrNSKGS1zVvYBCGJS1bMFji2qkuoTujGUqZZUbWV2QEl1QtCE")
api = tweepy.API(auth)
print(api.get_status(1324381354453250049, tweet_mode = "extended").full_text[0:-24])
print(api.get_status(1324381354453250049, tweet_mode = "extended").retweeted)


I’m holding a press conference with Dr. Scott Harris today to provide an update on our state’s #COVID19 efforts. You can watch it live at 11 a.m. via https://t.co/Y9juKjW5io or your local news stations. #alpolitics #TogetherAL @ALPublicHealth
False


In [2]:
# get names, parties, clean @s

import pandas as pd 
import numpy as np
dataset = pd.read_csv('handledatasetwithparty.csv', header=None)
dataset[3].unique()

array = dataset.to_numpy()
independents = array[array[:,3]=='I']
democrats = array[array[:,3]=='D']
republicans = array[array[:,3]=='R']
libertarians = array[array[:,3]=='L']
# other = array[array[:,3] not in ['I','D','R','L']] # doesn't work
other = array[(array[:,3]!='I') & (array[:,3]!='D') & (array[:,3]!='R') & (array[:,3]!='L')]

a = np.concatenate((republicans, democrats))
a = a[:, [3,4]]
# print np.where(array1==0, 1, array1) 
print(a[1][1])
print(np.any(a=='@RepFredKelle'))
print(np.any(a=='\xa0@SenatorLeahy'))
print(np.any(a=='@RepLloyd'))
a = np.where(a=='@RepLloyd', '@RepLloydDoggett', a)
a = np.where(a=='@RepFredKelle', '@RepFredKeller', a)
a = np.where(a=='\xa0@SenatorLeahy', '@SenatorLeahy', a)
print(np.any(a=='@RepLloyd'))
print(np.any(a=='xa0@SenatorLeahy'))
print(np.any(a=='@RepFredKelle'))

a
index = np.where(a=="@RepRickCrawford")
index
a[18]

@RepByrne
True
True
True
False
False
False


array(['R', '@RepRickCrawford'], dtype=object)

In [20]:
#get tweets. Instead of running this, can run cell 2 below instead

import tweepy

auth = tweepy.AppAuthHandler("Y49bfRjEyvHDa0vUBRFC4oQJy", "CCrNSKGS1zVvYBCGJS1bMFji2qkuoTujGUqZZUbWV2QEl1QtCE")
api = tweepy.API(auth)
failures = []
tmp=[]  
tmp2 = np.zeros(len(a))
def get_tweets(party, username): 
    
        
    auth = tweepy.AppAuthHandler("Y49bfRjEyvHDa0vUBRFC4oQJy", "CCrNSKGS1zVvYBCGJS1bMFji2qkuoTujGUqZZUbWV2QEl1QtCE")
    api = tweepy.API(auth)
  
    # 200 tweets to be extracted 
#     number_of_tweets=200
    try:
        tweets = api.user_timeline(screen_name=username, tweet_mode="extended") 

        tweets = [tweet for tweet in tweets] # CSV file created  
        for j in tweets: 

            # Appending tweets to the empty array tmp 
            text = j.full_text[0:-24]
            if text[0:4] != 'RT @':
                tmp.append([j.user._json['screen_name'],text,party])  
                print(j.user._json['screen_name'])
                print(j.full_text[0:-24])
    except:
        print('not authorized: ' + username)
        failures.append(username)
    # Empty Array 

    # create array of tweet information: username,  
    # tweet id, date/time, text 
        
    # Printing the tweets 
#     print(tmp) 
for user_id in a:
# user_id = ['R', "@RepRickCrawford"]
    get_tweets(user_id[0], user_id[1])
# tmp

SenShelby
Thank you to each and every one of the brave men and women who put their lives on the line to protect and defend our freedoms. I commend their strength and courage and am grateful for their dedication and selfless service to our nation. #VeteransDay
SenShelby
I’d like to wish the @USMC a happy 245th birthday. Their unwavering commitment and sacrifice for our great country is an inspiration to us all. We’re grateful for their service today and every day.
SenShelby
Our Republic relies on the integrity of our electoral system. Both parties should want fair and honest elections. Fraud, and the perception of fraud, erodes confidence and is damaging to the freedoms and liberties envisioned and establi
SenShelby
Time after time, we have demonstrated our willingness to work together and get the job done. We have before us the opportunity to deliver for the American people once again.
SenShelby
By &amp; large, these bills are the product of bipartisan cooperation among members of the 

RepMikeRogersAL
I would say this is unbelievable, but unfortunately, Nancy Pelosi’s attempts to weaken our great Congress should be expected at this point.
RepMikeRogersAL
This Veterans Day, join me in thanking all who have served. Without you and your loved ones’ sacrifices, America would be much less free and full of opportunity. We thank you #VeteransDay
RepMikeRogersAL
#HappyBirthdayMarines!
RepMikeRogersAL
Looking forward to it!
RepMikeRogersAL
Alabama does elections right. Every one of the battleground states should do themselves and our nation a favor and call
RepMikeRogersAL
@realDonaldTrump is absolutely right to challenge these results through all relevant legal means.This is as much about 2020 as it 
RepMikeRogersAL
Unfortunately, some states are utterly incompetent at running an election. Many Americans will view the 2020 election in doubt because of the operations of these states. And that will only undermine the integrity in our ins
RepMikeRogersAL
Great job by my colleag

SenDanSullivan
On this #VeteransDay, we thank the brave men and women who answered the call to serve and protect our country, including the many thousands of #veterans we are so fortunate to have in Alaska, and their family members. 🇺🇸
SenDanSullivan
@Sen_JoeManchin Thousands of America’s finest have served in the vicinity of #burnpits, leading in many cases to health complications years down the road. With the sobering lesson of Agent Orange &amp; Blue Water Navy Vietnam #veterans in mind, we will not wait decades to ta
SenDanSullivan
As we approach #VeteransDay, @Sen_JoeManchin and I are urging our Senate colleagues to pass S.2950, The #Veterans #BurnPits Exposure Recognition Act, our bill to recognize and concede exposure to toxic substances for veterans who served near burn pits.
SenDanSullivan
For 245 years, the @USMC—the most fearsome part of the greatest fighting force in human history—has defended America and stood up for freedom in all corners of the world. To all of my fellow

repdonyoung
Please join me in wishing a very Happy Birthday to @SenDanSullivan! Dan, it’s an honor to fight for Alaska with you, and I look forward to our work ahead next Congress.
repdonyoung
I have tested positive for COVID-19. I am feeling strong, following proper protocols, working from home in Alaska, and ask for privacy at this tim
repdonyoung
I'm proud to join my friend, @DorisMatsui, as we introduce the Japanese American Confinement Education Act. This bill takes crucial steps to educate younger generations about the horrors of Japanese internment, so that it will never happen again. More:
repdonyoung
In 1988, I was proud to support the Civil Liberties Act, which took crucial steps to acknowledge our nation's wrongdoing and to recommit ourselves to never letting racially-based incarceration happen again. Pictured in this thread is the original bill sig
repdonyoung
The forced internment of our fellow citizens during WWII is one of the darkest chapters in our history. The fact th

RepDavid
Wishing a bright and happy #Diwali to those celebrating this week. ✨
RepDavid
Exciting news - @Walmart will begin deploying autonomous vehicles for delivery starting next year in Scottsdale. Arizona continues to show leadership in implementing innovative technologies, and I look forward to seeing what comes next.
RepDavid
In just one month, we exceeded jobs report expectations once again. 

✅638,000 jobs added 
✅6.9% unemployment rate

This is a sign of a recovering labor force! 📈
RepDavid
I always enjoy seeing my staff take trips to further explore Arizona’s natural wonders. I wanted to share a picture of Vermillion Cliffs from the top of the Grand Canyon. 🌄🌵
RepDavid
Olivia and I took a trip to get our flu shots! 💉
 
It’s not too late for you to get a flu shot. Find a location near you here: https://t.co/Gwt1b28EXy
RepDavid
I was honored to be present for a local #VeteransDay celebration today. We will never forget the sacrifices made by all of our men and women in uniform. 

...

RepRickLarsen
...but it is a necessary action along with passing another #COVID19 relief bill to ease the burden on families and businesses. Relief needs to have focus on communities that the pandemic has hit dispropo
RepRickLarsen
.@SnoCo_DEM a tough decision.  I hear the frustration from people I represent about restrictions coupled with the frustration that still too many people aren’t doing the basics of mask-wearing and practicing social distancing...
RepRickLarsen
2/Congress needs to act on pandemic relief soon. Local restaurant workers and owners and others need assistance. Child care funding is critical. Funding local health dept and district workers to do more testing and for gi
RepRickLarsen
1/Congress can ease the burden families and businesses will continue to carry due to the potential of new pandemic restrictions as cases surge/ #covid19 hospitalizations increase. The public health response lea
RepRickLarsen
@joshtpm We will alwa
RepRickLarsen
Happy #Diwali to all those c

RepKimSchrier
Now it is Congress’ job to make sure that our small business owners and their employees are made whole. Squashing this virus is in our control, so let’s 
RepKimSchrier
This was a wise, necessary, and science-based decision. Most current transmission of COVID-19 has been at indoor gatherings, restaurants, and bars. It is a smart statement of our values to prioritize getting our children back to in person school.
RepKimSchrier
If you or a loved one are a Vietnam-era Veteran, I would love to meet you and present you with a commemorative pin. Sign up here:
RepKimSchrier
This week I met with Vietnam-era Veterans Doug in Kent, and Damian and Ted in Sammamish to thank them for their service. I am honored to be an official partner of the 50th Anniversary Commemoration of the Vietnam War and holding these events is one of my favorite parts of my job.
RepKimSchrier
One of my favorite events as a Congresswoman has been holding pinning ceremonies for Vietnam-era Veterans. A few weeks

Sen_JoeManchin
Wearing a face covering doesn’t just protect those around you from #COVID19 – it protects you. What’s your favorite mask? SHARE a photo with the hashtag #MaskUpWV. We must all do our part to slow the spread by staying 6 feet apart &amp; wearing a mask in public.
Sen_JoeManchin
ICYMI: We have got to get back to the basics. Rural West Virginians &amp; Americans want good-paying jobs, quality education &amp; affordable healthcare. These are the things Democrats are fighting for. This is not some radical agenda - this is an agenda that puts working Americans first
Sen_JoeManchin
The West Virginia Music Hall of Fame’s 2020 Induction ceremony is going virtual this year. Watch tonight from 7PM-9PM on your local @wvpublic station or on https://t.co/4nvYo7ueLz as we celebrate those who have made an incredible impa
Sen_JoeManchin
Fifty years ago, the devastating loss of 75 people, including members of the Marshall football team and Huntington community, shook us to our core. The u

RepBryanSteil
For important updates from my office, you can sign up for my newsletter here: https://t.co/Eo23rCIAgP
RepBryanSteil
Last year, I visited Walter Reed. The doctors, nurses, and health care professionals at Walter Reed do amazing work for our nation’s warriors. I had the honor of meeting a hero who was recovering at Walter Reed. We must never forget the sacrifice of our nation’s service members.
RepBryanSteil
When a coronavirus outbreak occurred at the Union Grove veterans home, I called for a federal investigation to prevent further cases. Our veterans &amp; frontline health care workers must be protected. I’ll continue demanding action. More from @TMJ4:
RepBryanSteil
Great news! Proud to work with the Wisconsin delegation to name the next Columbia Class Submarine in honor of our great state.

RepBryanSteil
The rise in COVID cases &amp; deaths in WI’s state-run veterans homes is concerning. I called for a VA investigation into the COVID outbreak in Union Grove. Our veterans

RepGwenMoore
Wishing a happy Diwali to 
RepGwenMoore
Child lead blood testing has decreased during the pandemic. I am calling on the CDC to find ways to ensure that children and families can get this vital testing so families can address the source of exposure and receive the services they need.
RepGwenMoore
COVID cases are soaring &amp; so is demand for COVID convalescent plasma – a new investigational therapy being used by hospitals to treat patients. If you’ve previously tested positive for COVID, please consider donating your plasma. Visit 
RepGwenMoore
The IRS must create a process that allows survivors to receive their funds, which can be used to build a be
RepGwenMoore
Because of financial abuse and economic power imbalances many domestic abuse survivors haven't received their stimulus check and aren't able to safely obtain them.
RepGwenMoore
Happy Veterans Day and thank you to all who have served our country. In showing our deep appreciation for their sacrifices, we must have a

In [63]:
#convert to df

origdf = pd.DataFrame(tmp, columns=["account","text","party"])
origdf.to_csv("origdataset.csv")
origdf

,account,text,party
0,SenShelby,Thank you to each and every one of the brave m...,R
1,SenShelby,I’d like to wish the @USMC a happy 245th birth...,R
2,SenShelby,Our Republic relies on the integrity of our el...,R
3,SenShelby,"Time after time, we have demonstrated our will...",R
4,SenShelby,"By &amp; large, these bills are the product of...",R
...,...,...,...
8847,RepGwenMoore,Joe Biden will follow the science and data to ...,D
8848,RepGwenMoore,I am so excited for the future of our country....,D
8849,RepGwenMoore,"For years, Republicans have used disingenuous ...",D
8850,RepGwenMoore,The voice of the American people is a powerful...,D


In [64]:
origdf = pd.read_csv('origdataset.csv')
origdf

,Unnamed: 0,account,text,party
0,0,SenShelby,Thank you to each and every one of the brave m...,R
1,1,SenShelby,I’d like to wish the @USMC a happy 245th birth...,R
2,2,SenShelby,Our Republic relies on the integrity of our el...,R
3,3,SenShelby,"Time after time, we have demonstrated our will...",R
4,4,SenShelby,"By &amp; large, these bills are the product of...",R
...,...,...,...,...
8847,8847,RepGwenMoore,Joe Biden will follow the science and data to ...,D
8848,8848,RepGwenMoore,I am so excited for the future of our country....,D
8849,8849,RepGwenMoore,"For years, Republicans have used disingenuous ...",D
8850,8850,RepGwenMoore,The voice of the American people is a powerful...,D


In [43]:
import nltk
nltk.download('stopwords')

len(tmp)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\quincy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


8852

In [44]:
# process text
import string

removable_punctuation = "!\"$%&'()*+,-./:;<=>?[\]^_`{|}~"
translator = str.maketrans('', '', removable_punctuation)

def remove_links(text):
    index = text.find('https://t.co/')
    if index is -1:
        return text
    else:
        text = text[0:index] + text[index+23:]
        return remove_links(text)
 

def preprocessor(text):
    text = remove_links(text)
#         print(text[index+23:])
    text = text.replace("&amp;", '').lower()
    return text.translate(translator)
    
# preprocessor(df.iloc[0]['text'])
df = origdf.copy()
df['text'] = df['text'].apply(preprocessor)

df

,account,text,party
0,SenShelby,thank you to each and every one of the brave m...,R
1,SenShelby,i’d like to wish the @usmc a happy 245th birth...,R
2,SenShelby,our republic relies on the integrity of our el...,R
3,SenShelby,time after time we have demonstrated our willi...,R
4,SenShelby,by large these bills are the product of bipar...,R
...,...,...,...
8847,RepGwenMoore,joe biden will follow the science and data to ...,D
8848,RepGwenMoore,i am so excited for the future of our country ...,D
8849,RepGwenMoore,for years republicans have used disingenuous f...,D
8850,RepGwenMoore,the voice of the american people is a powerful...,D


In [48]:
#tokenizer

from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

porter = PorterStemmer()

def tokenizer(text):
    return [porter.stem(word) for word in text.split() if word not in stopwords.words('english')]



['run', 'run', 'run']

In [59]:
#simple logistic regression

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)

X = list(df['text'])
y = list(df['party'])
X_train, X_test, y_train, y_test = train_test_split(
    X,y, test_size = 0.3, random_state = 0)

lr = Pipeline([('vect', tfidf),('clf', LogisticRegression(random_state=0, solver='liblinear'))])

lr.fit(X_train,y_train)
results_train = lr.predict(X_train) == y_train
print("Accuracy on training set:")
print(np.count_nonzero(results_train)/len(results_train))
results_test = lr.predict(X_test) == y_test
print("Accuracy on test set:")
print(np.count_nonzero(results_test)/len(results_test))

Accuracy on training set:
0.8863783085861846
Accuracy on test set:
0.7466114457831325
